In [1]:
#!pip install pip install TA-Lib
#!python3 -m pip install eodhd -U
#!pip install deap

In [2]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy
import os
import urllib
import json
import requests
from io import StringIO

from eod import EodHistoricalData
import pandas_datareader as pdr
import yfinance as yf
#import talib

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [5]:
#api = APIClient("618f834b7bfe27.18132752")
api_key = "618f834b7bfe27.18132752"

In [7]:
#from eodhd import APIClient
from eod import EodHistoricalData

In [9]:
client = EodHistoricalData(api_key)

In [71]:
symbol='TSLA.US'

In [72]:
resp = client.get_fundamental_equity(symbol)

In [73]:
resp['Highlights']

{'MarketCapitalization': 711156170752,
 'MarketCapitalizationMln': 711156.1708,
 'EBITDA': 15152000000,
 'PERatio': 71.9325,
 'PEGRatio': 1.2812,
 'WallStreetTargetPrice': 225.82,
 'BookValue': 16.818,
 'DividendShare': 0,
 'DividendYield': 0.0183,
 'EarningsShare': 3.11,
 'EPSEstimateCurrentYear': 3.14,
 'EPSEstimateNextYear': 3.89,
 'EPSEstimateNextQuarter': 0.85,
 'EPSEstimateCurrentQuarter': 0.72,
 'MostRecentQuarter': '2023-09-30',
 'ProfitMargin': 0.1121,
 'OperatingMarginTTM': 0.0756,
 'ReturnOnAssetsTTM': 0.0797,
 'ReturnOnEquityTTM': 0.2246,
 'RevenueTTM': 95924002816,
 'RevenuePerShareTTM': 30.277,
 'QuarterlyRevenueGrowthYOY': 0.088,
 'GrossProfitTTM': 20853000000,
 'DilutedEpsTTM': 3.11,
 'QuarterlyEarningsGrowthYOY': -0.442}

In [74]:
df = pd.DataFrame.from_dict(resp['Financials'])

In [75]:
df_bal = pd.DataFrame.from_dict(resp['Financials']['Balance_Sheet']['quarterly'])

In [76]:
df_cf = pd.DataFrame.from_dict(resp['Financials']['Cash_Flow']['quarterly'])

In [77]:
df_inc = pd.DataFrame.from_dict(resp['Financials']['Income_Statement']['quarterly'])

In [78]:
pd.concat([df_bal, df_cf, df_inc])

,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,...,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-03-31,2007-12-31,2007-03-31
date,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,...,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-03-31,2007-12-31,2007-03-31
filing_date,2023-10-23,2023-07-24,2023-04-24,2023-01-31,2022-10-24,2022-07-25,2022-04-25,2022-02-07,2021-10-25,2021-07-27,...,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-12-31,2008-03-31,2008-01-31,2007-03-31
currency_symbol,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,None,None,None
totalAssets,93941000000.00,90591000000.00,86833000000.00,82338000000.00,74426000000.00,68513000000.00,66038000000.00,62131000000.00,57834000000.00,55146000000.00,...,145320000.00,130424000.00,155916000.00,155916000.00,32606000.00,51699000.00,51699000.00,12924750.00,12924750.00,8709250.00
intangibleAssets,625000000.00,649000000.00,583000000.00,593000000.00,446000000.00,459000000.00,1515000000.00,1517000000.00,269000000.00,283000000.00,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
totalOtherIncomeExpenseNet,37000000.00,538000000.00,136000000.00,82000000.00,-52000000.00,10000000.00,23000000.00,-71000000.00,-122000000.00,-19000000.00,...,-3403000.00,-1088000.00,-543000.00,-2772000.00,586000.00,-2250000.00,-1194000.00,-1045250.00,-1045250.00,471500.00
discontinuedOperations,None,None,None,None,None,None,None,None,None,None,...,0.00,0.00,0.00,0.00,0.00,0.00,None,None,None,None
netIncomeFromContinuingOps,1878000000.00,2614000000.00,2539000000.00,3707000000.00,3331000000.00,2269000000.00,3280000000.00,2343000000.00,1659000000.00,1178000000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,None,None,None,None
netIncomeApplicableToCommonShares,1853000000.00,2703000000.00,2513000000.00,3722000000.00,3292000000.00,2259000000.00,3318000000.00,2321000000.00,1618000000.00,1142000000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,None,None,None,None
